In [18]:
!pip install -q yfinance
!pip install -q matplotlib
!pip install -q pandas_datareader
!pip install -q lightgbm
!pip install scikit-learn
!pip install lightgbm --upgrade



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\theaw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\theaw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\theaw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [422]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import numpy as np
import datetime
import lightgbm
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split


# Ecconomy Sample Stock

In [423]:
def stock_info(macro_indicators):
    ticker_data_info = pd.DataFrame()
    tick = yf.download(ticker, start = start_date_with_shift, end = end_date)
    ticker_data_info['average_stock_price'] = tick['Close'].resample(period_length).mean()
    ticker_data_info['average_stock_price_change'] = ticker_data_info['average_stock_price'] - ticker_data_info['average_stock_price'].shift(1)
    ticker_data_info['lag_average_stock_price_change'] = ticker_data_info['average_stock_price_change'].shift(1)
    ticker_data_info.reset_index(inplace=True)
    macro_indicators = pd.merge(macro_indicators, ticker_data_info[['Date', 'average_stock_price', 'average_stock_price_change', 'lag_average_stock_price_change']], 
                                left_on = 'DATE',
                                right_on = 'Date',
                                how = 'left')
    macro_indicators.drop(columns=['Date'], inplace=True)
    
    return macro_indicators


# Money Supply

In [424]:
def fetch_money_supply():
    # Define the list of money supply series to fetch
    money_supply_series = ['M1SL', 'M2SL']

    # Fetch money supply data using pandas_datareader
    money_supply_data = pdr.get_data_fred(money_supply_series, start_date_with_shift, end_date)

    # Calculate the YoY percentage change
    money_supply_data['M1SL_YoY'] = money_supply_data['M1SL'].pct_change(12) * 100
    money_supply_data['M2SL_YoY'] = money_supply_data['M2SL'].pct_change(12) * 100


    return money_supply_data


In [425]:
# Retrieves all MS between start and end date and returns MS differences in each period bin
def money_supply_info(macro_indicators):
    money_supply_data = fetch_money_supply()
    
    money_supply_data_average = money_supply_data.resample(period_length).mean()

    money_supply_data_average['M1-M2'] = money_supply_data_average['M1SL_YoY'] - money_supply_data_average['M2SL_YoY']

    macro_indicators = pd.merge(macro_indicators, money_supply_data_average['M1-M2'], on='DATE', how='left')
    return macro_indicators

# CPI Inflation

In [426]:
def fetch_cpi_annual_change():
    try:
        # Fetch U.S. Consumer Price Index (CPI) data from FRED
        cpi_data = pdr.get_data_fred('CPIAUCSL', start = start_date_with_shift, end = end_date)

        if not cpi_data.empty:
            # Calculate annual percentage change
            cpi_data['Annual_Change'] = cpi_data['CPIAUCSL'].pct_change(12) * 100

            return cpi_data[['Annual_Change']]
        else:
            return None  # Return None if there is no data
    except Exception as e:
        return None  # Return None if there is an error


In [427]:
def cpi_info(macro_indicators):
    cpi_data_info = pd.DataFrame()
    cpi_data = fetch_cpi_annual_change()
    cpi_data_info['average_cpi_change'] = cpi_data.resample(period_length).mean()
    cpi_data_info['single_average_cpi_change'] = cpi_data_info['average_cpi_change'] - cpi_data_info['average_cpi_change'].shift(1)
    cpi_data_info['lag_single_average_cpi_change'] = cpi_data_info['single_average_cpi_change'].shift(1)

    macro_indicators = pd.merge(macro_indicators, cpi_data_info[['average_cpi_change', 'single_average_cpi_change', 'lag_single_average_cpi_change']], on = 'DATE', how = 'left')
    
    return(macro_indicators)

# Unemployment

In [428]:
def fetch_unemployment_data():
    try:
        # Define the ticker symbol for the U.S. unemployment rate (UNRATE)
        unemployment_ticker = "UNRATE"

        # Fetch unemployment data from FRED
        unemployment_data = pdr.get_data_fred(unemployment_ticker, start = start_date_with_shift, end = end_date)

        return unemployment_data
    except Exception as e:
        return None  # Return None if there is an error

In [429]:
def unemployment_info(macro_indicators):
    unemployment_info = pd.DataFrame()
    unemployment_data = fetch_unemployment_data()
    unemployment_info['average_unemployment'] = unemployment_data.resample(period_length).mean()
    unemployment_info['average_unemployment_change'] = unemployment_info['average_unemployment'] - unemployment_info['average_unemployment'].shift(1)
    unemployment_info['lag_average_unemployment_change'] = unemployment_info['average_unemployment_change'].shift(1)
    macro_indicators = pd.merge(macro_indicators, unemployment_info[['average_unemployment', 'average_unemployment_change', 'lag_average_unemployment_change']], on = 'DATE', how = 'left')
    return macro_indicators

# Interest Rates

In [430]:
def fetch_fed_interest_rate():
    try:
        # Fetch Federal Reserve interest rate data (FEDFUNDS) from FRED
        fed_interest_rate = pdr.get_data_fred('FEDFUNDS', start = start_date_with_shift, end=end_date)

        if not fed_interest_rate.empty:
            return fed_interest_rate
        else:
            return None  # Return None if there is no data
    except Exception as e:
        return None  # Return None if there is an error



In [431]:
def fed_interest_rate_info(macro_indicators):
    fed_interest_rate_info = pd.DataFrame()
    fed_interest_rate = fetch_fed_interest_rate()
    fed_interest_rate_info['average_interest_rate'] = fed_interest_rate.resample(period_length).mean()
    fed_interest_rate_info['single_rate_period_change'] = fed_interest_rate_info['average_interest_rate'] - fed_interest_rate_info['average_interest_rate'].shift(1)
    fed_interest_rate_info['lag_single_rate_change'] = fed_interest_rate_info['single_rate_period_change'].shift(1)
    macro_indicators = pd.merge(macro_indicators, fed_interest_rate_info[['average_interest_rate','single_rate_period_change', 'lag_single_rate_change']],on = 'DATE', how = 'left')
    return(macro_indicators)



# CPI, Inflation, unemployment, interest rates


In [432]:
# New Function that takes the df(macro_indicators), function name, type of indicator
# Not currently used in my code yet but more efficent coding :)
def calculate_indicator_info(macro_indicators, fetch_function, column_prefix):
    indicator_info = pd.DataFrame()
    indicator_data = fetch_function()
    average_column_name = f'average_{column_prefix}'
    single_change_column_name = f'single_{column_prefix}_change'
    lag_single_change_column_name = f'lag_{single_change_column_name}'

    indicator_info[average_column_name] = indicator_data.resample(period_length).mean()
    indicator_info[single_change_column_name] = indicator_info[average_column_name] - indicator_info[average_column_name].shift(1)
    indicator_info[lag_single_change_column_name] = indicator_info[single_change_column_name].shift(1)

    macro_indicators = pd.merge(
        macro_indicators,
        indicator_info[[average_column_name, single_change_column_name, lag_single_change_column_name]],
        on='DATE',
        how='left'
    )

    return macro_indicators

In [433]:
def apply_indicators_info(macro_indicators, indicator_functions):
    for indicator_function in indicator_functions:
        macro_indicators = indicator_function(macro_indicators)
    return macro_indicators

# 

# MAIN

In [434]:
# Main
def get_data(end_date):
    period_length = '1M'
    ticker = "SPY"

    # Not sure why i can only do 36 months ahead
    start_date = end_date - relativedelta(months = 36)
    start_date_with_shift = start_date - relativedelta(months = 15)


    date_range = pd.date_range(start=start_date, end=end_date, freq = test_period_length)
    macro_indicators = pd.DataFrame({'DATE': date_range})

    stock_info(macro_indicators)

    indicator_functions = [stock_info, money_supply_info, fed_interest_rate_info, cpi_info, unemployment_info]
    macro_indicators = apply_indicators_info(macro_indicators, indicator_functions)
    return macro_indicators





In [435]:
end_date = datetime.datetime(2022, 1, 1)
macro_indicators = get_data(end_date)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


# BACKTESTING

### SELL, HOLD, BUY


In [436]:
# Determine SELL, HOLD, or BUY based on future change
def buy_or_sell(training_macro_indicators):
    training_macro_indicators['future_change'] = training_macro_indicators['average_stock_price'].shift(-1) - training_macro_indicators['average_stock_price']

    conditions = [
        (training_macro_indicators['future_change'] >= 5),
        (training_macro_indicators['future_change'] < 5) & (training_macro_indicators['future_change'] > -5),
        (training_macro_indicators['future_change'] <= -5)
    ]

    choices = ['BUY', 'HOLD', 'SELL']

    training_macro_indicators['buy_or_sell'] = np.select(conditions, choices, default='HOLD')

    return training_macro_indicators

In [437]:
train_macro_indicators = macro_indicators

In [438]:
full_train_macro_indicators = buy_or_sell(train_macro_indicators)
full_train_macro_indicators.tail()

,DATE,average_stock_price,average_stock_price_change,lag_average_stock_price_change,M1-M2,average_interest_rate,single_rate_period_change,lag_single_rate_change,average_cpi_change,single_average_cpi_change,lag_single_average_cpi_change,average_unemployment,average_unemployment_change,lag_average_unemployment_change,future_change,buy_or_sell
31,2021-08-31,444.520000,9.425718,12.109738,3.279482,0.09,-0.01,0.02,5.188292,-0.033214,-0.060105,5.2,-0.2,-0.5,-0.966189,HOLD
32,2021-09-30,443.553811,-0.966189,9.425718,2.929079,0.08,-0.01,-0.01,5.383630,0.195338,-0.033214,4.8,-0.4,-0.2,1.171426,HOLD
33,2021-10-31,444.725237,1.171426,-0.966189,2.683967,0.08,0.00,-0.01,6.237754,0.854124,0.195338,4.5,-0.3,-0.4,21.030478,BUY
34,2021-11-30,465.755716,21.030478,1.171426,2.509540,0.08,0.00,0.00,6.862388,0.624634,0.854124,4.2,-0.3,-0.3,0.649286,HOLD
35,2021-12-31,466.405002,0.649286,21.030478,2.224443,0.08,0.00,0.00,7.194459,0.332071,0.624634,3.9,-0.3,-0.3,NaN,HOLD


### LGBM MODEL

In [376]:
# Create a test data set from another period in time
# test_end_date = datetime.datetime(2010, 1, 1)
# test_macro_indicators = get_data(test_end_date)
# full_test_macro_indicators = buy_or_sell(test_macro_indicators)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,DATE,average_stock_price,average_stock_price_change,lag_average_stock_price_change,M1-M2,average_interest_rate,single_rate_period_change,lag_single_rate_change,average_cpi_change,single_average_cpi_change,lag_single_average_cpi_change,average_unemployment,average_unemployment_change,lag_average_unemployment_change,future_change,buy_or_sell
0,2019-01-31,260.104284,3.352180,-15.455517,-1.456675,2.40,0.13,0.07,1.550677,-0.451704,-0.144948,4.0,0.1,0.1,15.130980,BUY
1,2019-02-28,275.235265,15.130980,3.352180,-0.186713,2.40,0.00,0.13,1.520064,-0.030613,-0.451704,3.8,-0.2,0.1,4.795209,HOLD
2,2019-03-31,280.030474,4.795209,15.130980,-1.957033,2.41,0.01,0.00,1.853136,0.333072,-0.030613,3.8,0.0,-0.2,9.734288,BUY
3,2019-04-30,289.764762,9.734288,4.795209,-0.562198,2.42,0.01,0.01,1.991791,0.138656,0.333072,3.6,-0.2,0.0,-4.462489,HOLD
4,2019-05-31,285.302272,-4.462489,9.734288,-0.581499,2.39,-0.03,0.01,1.793518,-0.198273,0.138656,3.7,0.1,-0.2,3.681227,HOLD


In [415]:
var_columns = [c for c in full_training_macro_indicators if c not in['buy_or_sell']]

X = full_training_macro_indicators.loc[:, var_columns]
y = full_training_macro_indicators.loc[:, 'buy_or_sell']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)

In [416]:
train_data = lightgbm.Dataset(X_train, label = y_train)
valid_data = lightgbm.Dataset(X_valid, label = y_valid)

In [450]:
params = {
    'objective': 'multiclass',  # or 'binary' for binary classification
    'num_class': 3,    # Specify the number of classes for multiclass classification
    'metric': 'multi_logloss',  # Logarithmic Loss for classification
}

model = lightgbm.train(
    params, 
    train_data, 
    valid_sets=valid_data,
    num_boost_round=100 
)

# Make predictions on new data
predictions = model.predict(X_valid)

# Convert predicted probabilities to class labels
predicted_labels = predictions.argmax(axis=1)

# Evaluate the model
accuracy = accuracy_score(y_valid, predicted_labels)
print(f'Accuracy: {accuracy}')

ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: DATE: datetime64[ns]